# Création table nettoyage chargement

In [2]:
import pandas as pd

from sqlalchemy import create_engine
from azure.identity import DefaultAzureCredential
from azure.appconfiguration import AzureAppConfigurationClient

import pandas as pd
from sodapy import Socrata

## 2. Import du dataset connection api seattle data

In [8]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.seattle.gov", None)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("2bpz-gwpy", limit=5000)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [9]:
df.shape

(3376, 45)

In [11]:
df.head(3)

,osebuildingid,datayear,buildingtype,primarypropertytype,propertyname,address,city,state,zipcode,taxparcelidentificationnumber,councildistrictcode,neighborhood,latitude,longitude,yearbuilt,numberofbuildings,numberoffloors,propertygfatotal,propertygfaparking,propertygfabuilding_s,listofallpropertyusetypes,largestpropertyusetype,largestpropertyusetypegfa,energystarscore,siteeui_kbtu_sf,siteeuiwn_kbtu_sf,sourceeui_kbtu_sf,sourceeuiwn_kbtu_sf,siteenergyuse_kbtu,siteenergyusewn_kbtu,steamuse_kbtu,electricity_kwh,electricity_kbtu,naturalgas_therms,naturalgas_kbtu,defaultdata,compliancestatus,totalghgemissions,ghgemissionsintensity,secondlargestpropertyusetype,secondlargestpropertyuse,thirdlargestpropertyusetype,thirdlargestpropertyusetypegfa,yearsenergystarcertified,outlier
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101,0659000030,7,DOWNTOWN,47.6122,-122.33799,1927,1,12,88434,0,88434,Hotel,Hotel,88434,60,81.69999695,84.30000305,182.5,189,7226362.5,7456910,2003882,1156514.25,3946027,12764.5293,1276453,False,Compliant,249.98,2.83,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,1,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880,61,94.80000305,97.90000153,176.1000061,179.3999939,8387933,8664479,0,950425.1875,3242851,51450.81641,5145082,False,Compliant,295.86,2.86,Parking,15064,Restaurant,4622,NaN,NaN
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101,0659000475,7,DOWNTOWN,47.61393,-122.3381,1969,1,41,956110,196718,759392,Hotel,Hotel,756493,43,96,97.69999695,241.8999939,244.1000061,72587024,73937112,21566554,14515435,49526664,14938,1493800,False,Compliant,2089.28,2.19,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.columns

Index(['osebuildingid', 'datayear', 'buildingtype', 'primarypropertytype',
       'propertyname', 'address', 'city', 'state', 'zipcode',
       'taxparcelidentificationnumber', 'councildistrictcode', 'neighborhood',
       'latitude', 'longitude', 'yearbuilt', 'numberofbuildings',
       'numberoffloors', 'propertygfatotal', 'propertygfaparking',
       'propertygfabuilding_s', 'listofallpropertyusetypes',
       'largestpropertyusetype', 'largestpropertyusetypegfa',
       'energystarscore', 'siteeui_kbtu_sf', 'siteeuiwn_kbtu_sf',
       'sourceeui_kbtu_sf', 'sourceeuiwn_kbtu_sf', 'siteenergyuse_kbtu',
       'siteenergyusewn_kbtu', 'steamuse_kbtu', 'electricity_kwh',
       'electricity_kbtu', 'naturalgas_therms', 'naturalgas_kbtu',
       'defaultdata', 'compliancestatus', 'totalghgemissions',
       'ghgemissionsintensity', 'secondlargestpropertyusetype',
       'secondlargestpropertyuse', 'thirdlargestpropertyusetype',
       'thirdlargestpropertyusetypegfa', 'yearsenergystarcerti